## MLSD: PySpark Data Frames Employees
#### Maria Rafaela Alves Abrunhosa 107658
**24th March 2025**

Given the dataset *employees.csv* found in the shared folder, explore the following operations:
- Display schema and first few rows.
- Filter employees with a salary greater than 5000.
- Calculate the average salary per department.
- Find the employee with the highest salary in each department.
- Add a new column [Salary Increase] which increases salaries by 10% 
for employees with more than 5 years of experience.
- Sort the employees by experience in descending order and show the top 5.

In [27]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, rank, when, desc
import pandas as pd

In [3]:
# create a spark session
spark = SparkSession.builder.appName("Employees Analysis").getOrCreate()

# create a dataframe using employees.csv
path = "employees.csv"
df = spark.read.csv(path, header=True, inferSchema=True)
df.show()

25/04/01 12:27:23 WARN Utils: Your hostname, rafaela-XPS-15-9570 resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp59s0)
25/04/01 12:27:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 12:27:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1001|  Frank|        IT|  8231|                20|
|      1002|  Alice|        HR|  3716|                23|
|      1003|  Alice| Marketing|  9334|                11|
|      1004|  David|     Sales| 14785|                 8|
|      1005|   Kara|Operations|  8301|                13|
|      1006|Charlie|Operations|  5720|                28|
|      1007|    Eve|        IT|  4801|                 4|
|      1008|   Liam|   Finance|  9243|                15|
|      1009|    Bob|   Finance| 11775|                 5|
|      1010|    Bob|Operations|  6456|                27|
|      1011| Hannah|        IT|  6457|                15|
|      1012|Charlie|        HR| 11244|                29|
|      1013|  Alice| Marketing|  8875|                16|
|      1014|  Frank|Operations|  7422|                28|
|      1015|Ch

In [ ]:
# Display schema and first few rows.
df.printSchema()
df.show() # shows top 20 rows, we can choose the number

root
 |-- EmployeeID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience (Years): integer (nullable = true)

+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1001|  Frank|        IT|  8231|                20|
|      1002|  Alice|        HR|  3716|                23|
|      1003|  Alice| Marketing|  9334|                11|
|      1004|  David|     Sales| 14785|                 8|
|      1005|   Kara|Operations|  8301|                13|
|      1006|Charlie|Operations|  5720|                28|
|      1007|    Eve|        IT|  4801|                 4|
|      1008|   Liam|   Finance|  9243|                15|
|      1009|    Bob|   Finance| 11775|                 5|
|      1010|    Bob|Operations|  6456|                27|
|      1011| Hannah|        I

In [ ]:
# Filter employees with a salary greater than 5000.
salaryMore5000 = df.filter(df.Salary > 5000)
salaryMore5000.show()

+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1001|  Frank|        IT|  8231|                20|
|      1003|  Alice| Marketing|  9334|                11|
|      1004|  David|     Sales| 14785|                 8|
|      1005|   Kara|Operations|  8301|                13|
|      1006|Charlie|Operations|  5720|                28|
|      1008|   Liam|   Finance|  9243|                15|
|      1009|    Bob|   Finance| 11775|                 5|
|      1010|    Bob|Operations|  6456|                27|
|      1011| Hannah|        IT|  6457|                15|
|      1012|Charlie|        HR| 11244|                29|
|      1013|  Alice| Marketing|  8875|                16|
|      1014|  Frank|Operations|  7422|                28|
|      1016|  David|   Finance|  7087|                24|
|      1018|    Bob|     Sales| 11012|                17|
|      1019|  

In [ ]:
# Calculate the average salary per department.
averageSalaryDepartment = df.groupBy("Department").agg(avg("Salary").alias("Average Salary")) # with column renamed
averageSalaryDepartment.show()

+----------+-----------------+
|Department|   Average Salary|
+----------+-----------------+
|     Sales|8680.865030674846|
|        HR| 8596.44642857143|
|   Finance|9233.802259887005|
| Marketing|8708.290909090909|
|        IT|8790.344827586207|
|Operations|8879.686274509804|
+----------+-----------------+



In [17]:
# Find the employee with the highest salary in each department.

# employeeHighSalaryDepartment = df.groupBy("Department").agg(max("Salary").alias("Max Salary")) # after this we need to do a join to get the employees

# with window function is more eficient for large data volumes
window = Window.partitionBy("Department").orderBy(col("Salary").desc())

employeeHighSalaryDepartment = df.withColumn("rank", rank().over(window)).filter(col("rank") == 1).drop("rank")
employeeHighSalaryDepartment.show()

+----------+-----+----------+------+------------------+
|EmployeeID| Name|Department|Salary|Experience (Years)|
+----------+-----+----------+------+------------------+
|      1271|David|   Finance| 14928|                27|
|      1763|Frank|        HR| 14999|                29|
|      1638|Alice|        IT| 14859|                 2|
|      1953|Alice| Marketing| 14938|                30|
|      1924|  Eve|Operations| 14978|                29|
|      1607|  Bob|     Sales| 14974|                11|
+----------+-----+----------+------+------------------+



In [ ]:
# Add a new column [Salary Increase] which increases salaries by 10% for employees with more than 5 years of experience.
df = df.withColumn("Salary Increase", when(col("Experience (Years)") > 5, col("Salary") + (col("Salary") * 0.10)).otherwise(col("Salary")))
df.show()

+----------+-------+----------+------+------------------+---------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|Salary Increase|
+----------+-------+----------+------+------------------+---------------+
|      1001|  Frank|        IT|  8231|                20|         9054.1|
|      1002|  Alice|        HR|  3716|                23|         4087.6|
|      1003|  Alice| Marketing|  9334|                11|        10267.4|
|      1004|  David|     Sales| 14785|                 8|        16263.5|
|      1005|   Kara|Operations|  8301|                13|         9131.1|
|      1006|Charlie|Operations|  5720|                28|         6292.0|
|      1007|    Eve|        IT|  4801|                 4|         4801.0|
|      1008|   Liam|   Finance|  9243|                15|        10167.3|
|      1009|    Bob|   Finance| 11775|                 5|        11775.0|
|      1010|    Bob|Operations|  6456|                27|         7101.6|
|      1011| Hannah|        IT|  6457|

In [ ]:
# Sort the employees by experience in descending order and show the top 5.
experienceSort = df.orderBy(col("Experience (Years)").desc())
experienceSort.show(5)

+----------+-------+----------+------+------------------+---------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|Salary Increase|
+----------+-------+----------+------+------------------+---------------+
|      1090|  Alice| Marketing|  9467|                30|        10413.7|
|      1139|   Liam|     Sales|  5967|                30|         6563.7|
|      1111|    Ian|     Sales|  6763|                30|         7439.3|
|      1112|   Kara|        IT|  5926|                30|         6518.6|
|      1081|Charlie|     Sales| 10647|                30|        11711.7|
+----------+-------+----------+------+------------------+---------------+
only showing top 5 rows

